In [1]:
import os
import re

import pandas as pd

In [2]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

In [3]:
df_train.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...


In [4]:
df_test.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Te-1,8/18/2016,Bank account or service,"Account opening, closing, or management",Company has responded to the consumer and the ...,8/18/2016,No,XXXX / XXXX / 16 I called Citibank to open a c...
1,Te-2,4/18/2016,Debt collection,Communication tactics,Company believes it acted appropriately as aut...,4/20/2016,No,I'm struggling financially. I called and I off...
2,Te-3,3/23/2016,Credit reporting,Incorrect information on credit report,NaN,3/23/2016,No,"In XXXX of 2015, an automatic payment was conf..."
3,Te-4,6/26/2017,Student loan,Dealing with your lender or servicer,NaN,6/26/2017,NaN,"I submitted a request to XXXX, which is my cur..."
4,Te-5,5/13/2016,Credit reporting,Incorrect information on credit report,Company has responded to the consumer and the ...,5/13/2016,No,A state tax lien was filed against me XXXX / X...


### Shape of data

In [5]:
df_train.shape, df_test.shape

((43266, 9), (18543, 8))

### Renaming columns

In [6]:
def rename_columns(columns: list):
    columns = ['_'.join(re.sub('[.-]', '_', col).strip().lower().split()) for col in columns]
    return columns

In [7]:
df_train.columns = rename_columns(columns=df_train.columns.tolist())
df_test.columns = rename_columns(columns=df_test.columns.tolist())

### Unique value count

In [8]:
def unique_value_count(df):
    df_nunique = df.nunique().reset_index()
    df_nunique['dtype'] = df.dtypes.reset_index().loc[:, 0]
    df_nunique.columns = ['column', 'nunique', 'dtype']
    
    display(df_nunique)

In [9]:
unique_value_count(df=df_train)

,column,nunique,dtype
0,complaint_id,43266,object
1,date_received,920,object
2,transaction_type,18,object
3,complaint_reason,150,object
4,company_response,10,object
5,date_sent_to_company,930,object
6,complaint_status,5,object
7,consumer_disputes,2,object
8,consumer_complaint_summary,43022,object


In [10]:
unique_value_count(df=df_test)

,column,nunique,dtype
0,complaint_id,18543,object
1,date_received,914,object
2,transaction_type,18,object
3,complaint_reason,147,object
4,company_response,10,object
5,date_sent_to_company,914,object
6,consumer_disputes,2,object
7,consumer_complaint_summary,18485,object


In [11]:
# Dropping complaint_id column
df_train = df_train.drop(columns=["complaint_id"])
df_test = df_test.drop(columns=["complaint_id"])

In [12]:
# Changing the format of date columns
df_train["date_received"] = pd.to_datetime(df_train["date_received"], format="%m/%d/%Y")
df_train["date_sent_to_company"] = pd.to_datetime(df_train["date_sent_to_company"], format="%m/%d/%Y")

df_test["date_received"] = pd.to_datetime(df_test["date_received"], format="%m/%d/%Y")
df_test["date_sent_to_company"] = pd.to_datetime(df_test["date_sent_to_company"], format="%m/%d/%Y")

In [13]:
# Sorting the dataframes
df_train = df_train.sort_values(by=["date_received"])
df_test = df_test.sort_values(by=["date_received"])

In [14]:
# Defining column order
cols_order = [
    "date_received", "transaction_type", 
    "complaint_reason", "consumer_complaint_summary",
    "date_sent_to_company", "company_response",
    "consumer_disputes", "complaint_status"
]

df_train = df_train[cols_order]
df_test = df_test[cols_order[:-1]]

In [15]:
os.makedirs("../data/cleaned", exist_ok=True)

In [16]:
df_train.to_csv("../data/cleaned/train.csv", index=False)
df_test.to_csv("../data/cleaned/test.csv", index=False)